In [2]:
import tkinter as tk
from tkinter import ttk, filedialog
import cv2
import numpy as np
from rembg import remove
from PIL import Image, ImageTk
import requests
import joblib

def remove_background_with_rembg(image_path):
    original_image = cv2.imread(image_path)
    resized_image = cv2.resize(original_image, (1080, 1080))
    cv2.imwrite('resized_image.png', resized_image)
    input_image = Image.open('resized_image.png')
    output_image = remove(input_image)
    output_path = 'processed_image.png'
    output_image.save(output_path)
    processed_image = cv2.imread(output_path)
    return processed_image

def extract_features(image_path):
    preprocessed_image = remove_background_with_rembg(image_path)
    hsv_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2HSV)
    mean_hsv = np.mean(hsv_image, axis=(0, 1))
    mean_rgb = np.mean(preprocessed_image, axis=(0, 1))
    features = np.concatenate([mean_hsv, mean_rgb])
    return features, preprocessed_image

def process_prediction(image_path):
    image_features, preprocessed_image = extract_features(image_path)
    knn_model = joblib.load('knn_model.pkl')
    predicted_label = knn_model.predict([image_features])
    return predicted_label[0], preprocessed_image

def check_for_nutrient_deficiency(data): #CRUSIAL CODE UBAH KETIKA SELESAI TESTING
    npk_value = data['Nitrogen'] + data['Phosphorous'] + data['Potassium']
    if npk_value > 50 and data['ph'] < 7:
        return "Kekurangan Unsur Hara"
    else:
        return "Tidak ada kekurangan unsur hara"

def choose_image():
    file_path = filedialog.askopenfilename(title="Pilih Gambar", filetypes=[("Image files", "*.jpg;*.png;*.jpeg")])
    if file_path:
        predicted_label, processed_image = process_prediction(file_path)
        data = {'Nitrogen': 50, 'Phosphorous': 40, 'Potassium': 30, 'ph': 6.5}  # Contoh data Firebase
        nutrient_status = check_for_nutrient_deficiency(data)
        show_prediction(processed_image, predicted_label, nutrient_status)

def show_prediction(processed_image, predicted_label, nutrient_status):
    processed_img = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
    processed_img = Image.fromarray(processed_img)
    processed_img.thumbnail((600, 400))
    processed_img = ImageTk.PhotoImage(processed_img)

    processed_label.config(image=processed_img)
    processed_label.image = processed_img

    prediction_label.config(text=f"Prediksi: {predicted_label}")
    nutrient_label.config(text=f"Status Unsur Hara: {nutrient_status}")

def update_data_and_gui():
    global data_count, collected_data

    url = "https://tugasakhir-189d6-default-rtdb.asia-southeast1.firebasedatabase.app/data1.json"
    data = requests.get(url).json()

    numeric_values = [value for key, value in data.items() if key != 'Timestamp' and key in ['Nitrogen', 'Phosphorous', 'Potassium', 'ph']]
    
    collected_data.append(numeric_values)
    data_count += 1

    if data_count >= 10:
        averages = [sum(column) / len(column) for column in zip(*collected_data)]
        average_table.delete(*average_table.get_children())  # Hapus data sebelumnya
        average_table.insert("", "end", values=averages)  # Tambahkan data rata-rata yang baru
        collected_data = []
        data_count = 0
        data_table.delete(*data_table.get_children())

    for i in range(0, len(numeric_values), 4):
        data_table.insert("", "end", values=numeric_values[i:i+4])
        if len(data_table.get_children()) > 10:
            data_table.delete(data_table.get_children()[0])

    root.after(1000, update_data_and_gui)

root = tk.Tk()
root.title("Aplikasi Gabungan")

# Frame untuk bagian prediksi
prediction_frame = ttk.Frame(root)
prediction_frame.pack(side="left", padx=10, pady=10)

select_button = tk.Button(prediction_frame, text="Pilih Gambar", command=choose_image, bg="lightblue", font=("Helvetica", 14))
select_button.pack(pady=20)

processed_label = tk.Label(prediction_frame, text="Gambar Hasil Proses", font=("Helvetica", 16, "bold"))
processed_label.pack()

prediction_label = tk.Label(prediction_frame, text="Prediksi:", font=("Helvetica", 18, "bold"), fg="green")
prediction_label.pack(pady=20)

nutrient_label = tk.Label(prediction_frame, text="Status Unsur Hara:", font=("Helvetica", 18, "bold"), fg="red")
nutrient_label.pack(pady=20)

# Frame untuk bagian Firebase Data
firebase_frame = ttk.Frame(root)
firebase_frame.pack(side="right", padx=10, pady=10)

def create_table(parent, columns, anchor="w"):
    table = ttk.Treeview(parent, columns=columns, show="headings")
    for column in columns:
        table.heading(column, text=column, anchor=anchor)
        table.column(column, anchor=anchor)
    table.pack(padx=10, pady=10)
    return table

average_table = create_table(firebase_frame, ("R Nitrogen", "R Fosfor", "R Kalium", "R pH"), anchor="center")
data_table = create_table(firebase_frame, ("Nitrogen", "Fosfor", "Kalium", "pH"), anchor="center")

data_count = 0
collected_data = []

update_data_and_gui()

root.mainloop()


In [4]:
import tkinter as tk
from tkinter import ttk, filedialog
import cv2
import numpy as np
from rembg import remove
from PIL import Image, ImageTk
import requests
import joblib

def remove_background_with_rembg(image_path):
    original_image = cv2.imread(image_path)
    resized_image = cv2.resize(original_image, (1080, 1080))
    cv2.imwrite('resized_image.png', resized_image)
    input_image = Image.open('resized_image.png')
    output_image = remove(input_image)
    output_path = 'processed_image.png'
    output_image.save(output_path)
    processed_image = cv2.imread(output_path)
    return processed_image

def extract_features(image_path):
    preprocessed_image = remove_background_with_rembg(image_path)
    hsv_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2HSV)
    mean_hsv = np.mean(hsv_image, axis=(0, 1))
    mean_rgb = np.mean(preprocessed_image, axis=(0, 1))
    features = np.concatenate([mean_hsv, mean_rgb])
    return features, preprocessed_image

def process_prediction(image_path):
    image_features, preprocessed_image = extract_features(image_path)
    knn_model = joblib.load('knn_model.pkl')
    predicted_label = knn_model.predict([image_features])
    return predicted_label[0], preprocessed_image

def check_for_nutrient_deficiency(data):
    npk_value = data['Nitrogen'] + data['Phosphorous'] + data['Potassium']
    if npk_value > 50 and data['ph'] < 7:
        return "Kekurangan Unsur Hara"
    else:
        return "Tidak ada kekurangan unsur hara"

def choose_image():
    file_path = filedialog.askopenfilename(title="Pilih Gambar", filetypes=[("Image files", "*.jpg;*.png;*.jpeg")])
    if file_path:
        predicted_label, processed_image = process_prediction(file_path)
        data = {'Nitrogen': 50, 'Phosphorous': 40, 'Potassium': 30, 'ph': 6.5}  # Contoh data Firebase
        nutrient_status = check_for_nutrient_deficiency(data)
        show_prediction(processed_image, predicted_label, nutrient_status)

def open_camera():
    global cap
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Camera tidak dapat diakses")
        return
    
    show_camera()

def show_camera():
    ret, frame = cap.read()
    if ret:
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        camera_label.imgtk = imgtk
        camera_label.configure(image=imgtk)
        camera_label.after(10, show_camera)
    else:
        print("Error: Tidak dapat membaca frame dari kamera")

def capture_image():
    ret, frame = cap.read()
    if ret:
        image_path = "captured_image.png"
        cv2.imwrite(image_path, frame)
        predicted_label, processed_image = process_prediction(image_path)
        data = {'Nitrogen': 50, 'Phosphorous': 40, 'Potassium': 30, 'ph': 6.5}  # Contoh data Firebase
        nutrient_status = check_for_nutrient_deficiency(data)
        show_prediction(processed_image, predicted_label, nutrient_status)
        cap.release()
        camera_label.config(image='')  # Hapus tampilan kamera
    else:
        print("Error: Tidak dapat menangkap gambar")

def show_prediction(processed_image, predicted_label, nutrient_status):
    processed_img = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
    processed_img = Image.fromarray(processed_img)
    processed_img.thumbnail((600, 400))
    processed_img = ImageTk.PhotoImage(processed_img)

    processed_label.config(image=processed_img)
    processed_label.image = processed_img

    prediction_label.config(text=f"Prediksi: {predicted_label}")
    nutrient_label.config(text=f"Status Unsur Hara: {nutrient_status}")

def update_data_and_gui():
    global data_count, collected_data

    url = "https://tugasakhir-189d6-default-rtdb.asia-southeast1.firebasedatabase.app/data1.json"
    data = requests.get(url).json()

    numeric_values = [value for key, value in data.items() if key != 'Timestamp' and key in ['Nitrogen', 'Phosphorous', 'Potassium', 'ph']]
    
    collected_data.append(numeric_values)
    data_count += 1

    if data_count >= 10:
        averages = [sum(column) / len(column) for column in zip(*collected_data)]
        average_table.delete(*average_table.get_children())
        average_table.insert("", "end", values=averages)
        collected_data = []
        data_count = 0
        data_table.delete(*data_table.get_children())

    for i in range(0, len(numeric_values), 4):
        data_table.insert("", "end", values=numeric_values[i:i+4])
        if len(data_table.get_children()) > 10:
            data_table.delete(data_table.get_children()[0])

    root.after(1000, update_data_and_gui)

root = tk.Tk()
root.title("Aplikasi Gabungan")

prediction_frame = ttk.Frame(root)
prediction_frame.pack(side="left", padx=10, pady=10)

select_button = tk.Button(prediction_frame, text="Pilih Gambar", command=choose_image, bg="lightblue", font=("Helvetica", 14))
select_button.pack(pady=20)

capture_button = tk.Button(prediction_frame, text="Buka Kamera", command=open_camera, bg="lightgreen", font=("Helvetica", 14))
capture_button.pack(pady=20)

camera_label = tk.Label(prediction_frame)
camera_label.pack(pady=20)

snap_button = tk.Button(prediction_frame, text="Ambil Gambar", command=capture_image, bg="lightgreen", font=("Helvetica", 14))
snap_button.pack(pady=20)

processed_label = tk.Label(prediction_frame, text="Gambar Hasil Proses", font=("Helvetica", 16, "bold"))
processed_label.pack()

prediction_label = tk.Label(prediction_frame, text="Prediksi:", font=("Helvetica", 18, "bold"), fg="green")
prediction_label.pack(pady=20)

nutrient_label = tk.Label(prediction_frame, text="Status Unsur Hara:", font=("Helvetica", 18, "bold"), fg="red")
nutrient_label.pack(pady=20)

firebase_frame = ttk.Frame(root)
firebase_frame.pack(side="right", padx=10, pady=10)

def create_table(parent, columns, anchor="w"):
    table = ttk.Treeview(parent, columns=columns, show="headings")
    for column in columns:
        table.heading(column, text=column, anchor=anchor)
        table.column(column, anchor=anchor)
    table.pack(padx=10, pady=10)
    return table

average_table = create_table(firebase_frame, ("R Nitrogen", "R Fosfor", "R Kalium", "R pH"), anchor="center")
data_table = create_table(firebase_frame, ("Nitrogen", "Fosfor", "Kalium", "pH"), anchor="center")

data_count = 0
collected_data = []

update_data_and_gui()

root.mainloop()


c:\Users\PAIJO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


Error: Tidak dapat membaca frame dari kamera


c:\Users\PAIJO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


Error: Tidak dapat membaca frame dari kamera
